<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#What-attributes-have-been-discovered?" data-toc-modified-id="What-attributes-have-been-discovered?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>What attributes have been discovered?</a></span></li><li><span><a href="#Features" data-toc-modified-id="Features-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Features</a></span><ul class="toc-item"><li><span><a href="#Accessing-coordinate-variables" data-toc-modified-id="Accessing-coordinate-variables-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Accessing coordinate variables</a></span></li><li><span><a href="#Rewriting-arguments" data-toc-modified-id="Rewriting-arguments-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Rewriting arguments</a></span><ul class="toc-item"><li><span><a href="#Slicing" data-toc-modified-id="Slicing-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Slicing</a></span></li><li><span><a href="#Plotting" data-toc-modified-id="Plotting-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Plotting</a></span></li><li><span><a href="#Resample-&amp;-groupby" data-toc-modified-id="Resample-&amp;-groupby-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Resample &amp; groupby</a></span></li><li><span><a href="#Rolling-&amp;-coarsen" data-toc-modified-id="Rolling-&amp;-coarsen-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Rolling &amp; coarsen</a></span></li><li><span><a href="#miscellaneous-features" data-toc-modified-id="miscellaneous-features-2.2.5"><span class="toc-item-num">2.2.5&nbsp;&nbsp;</span>miscellaneous features</a></span></li></ul></li><li><span><a href="#Augmented-functionality" data-toc-modified-id="Augmented-functionality-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Augmented functionality</a></span><ul class="toc-item"><li><span><a href="#Cell-Measures" data-toc-modified-id="Cell-Measures-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Cell Measures</a></span></li></ul></li></ul></li></ul></div>

This is a brief introduction to `cf_xarray`'s capabilities.

In [ ]:
import cf_xarray
import numpy as np
import xarray as xr

Lets read two datasets.

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature")
ds

This one is inspired by POP model output and illustrates how the coordinates attribute is interpreted

In [ ]:
pop = xr.Dataset()
pop.coords["TLONG"] = (("nlat", "nlon"), np.ones((20, 30)), {"units": "degrees_east"})
pop.coords["TLAT"] = (("nlat", "nlon"), 2 * np.ones((20, 30)), {"units": "degrees_north"})
pop.coords["ULONG"] = (("nlat", "nlon"), 0.5 * np.ones((20, 30)), {"units": "degrees_east"})
pop.coords["ULAT"] = (("nlat", "nlon"), 2.5 * np.ones((20, 30)), {"units": "degrees_north"})

pop["UVEL"] = (
    ("nlat", "nlon"),
    np.ones((20, 30)) * 15,
    {"coordinates": "ULONG ULAT"},
)
pop["TEMP"] = (
    ("nlat", "nlon"),
    np.ones((20, 30)) * 15,
    {"coordinates": "TLONG TLAT"},
)


# What attributes have been discovered?

In [ ]:
ds.lon

`ds.lon` has attributes `axis: X`. This means that `cf_xarray` can identify the `'X'` axis as being represented by the `lon` variable.

It can also use the `standard_name` and `units` attributes to infer that `lon` is "Longitude". To see variable names that `cf_xarray` can infer, use `.cf.describe()`

In [ ]:
ds.cf.describe()

For `pop`, only `latitude` and `longitude` are detected, not `X` or `Y`. Please comment here: https://github.com/xarray-contrib/cf-xarray/issues/23 if you have opinions about this behaviour.

In [ ]:
pop.cf.describe()

# Features

## Accessing coordinate variables

`.cf` implements `__getitem__` to allow easy access to coordinate and axis variables.

In [ ]:
ds.cf["X"]

`Dataset.cf[...]` returns a `Dataset`, possibly with multiple variables

In [ ]:
pop.cf["longitude"]

`DataArray.cf[...]` returns a single `DataArray`, parsing the `coordinates` attribute if present, so we correctly get the `TLONG` variable and not the `ULONG` variable

In [ ]:
pop.TEMP.cf["longitude"]

## Rewriting arguments

`cf_xarray` can rewrite arguments for a large number of xarray functions. By this I mean that instead of specifing say `dim="lon"`, you can pass `dim="X"` or `dim="longitude"` and `cf_xarray` will rewrite that to `dim="lon"` based on the attributes present in the dataset. 

Here are a few examples

### Slicing

In [ ]:
ds.air.cf.isel(T=1)

### Plotting

In [ ]:
ds.air.cf.isel(time=1).cf.plot(x="X", y="Y")

In [ ]:
ds.air.cf.isel(T=1, Y=[0, 1, 2]).cf.plot(x="longitude", hue="latitude")

`cf_xarray` can facet

In [ ]:
ds.air.isel(time=slice(4)).cf.plot(x="longitude", y="latitude", col="time")

### Resample & groupby

In [ ]:
ds.cf.resample(T="D").mean()

In [ ]:
ds.cf.groupby("time.month").mean("longitude")

### Rolling & coarsen 

In [ ]:
ds.cf.rolling(X=5).mean()

coarsen works but eberything later will break because of xarray bug https://github.com/pydata/xarray/issues/4120

ds.isel(lon=slice(50)).cf.coarsen(Y=5, X=10).mean()

### miscellaneous features

You can mix "special names" and variable names

In [ ]:
ds.cf.groupby("time.month").mean(["lat", "X"])

## Augmented functionality

### Cell Measures

`cf_xarray` can weight by cell measure variables `"area"` and `"volume"` if the appropriate attribute is set

In [ ]:
# Lets make some weights (not sure if this is right)
ds.coords["cell_area"] = (
    np.cos(ds.air.cf["latitude"] * np.pi / 180)
    * xr.ones_like(ds.air.cf["longitude"])
    * 105e3
    * 110e3
)
# and set proper attributes
ds.air.attrs["cell_measures"] = "area: cell_area"

In [ ]:
ds.air.cf.weighted("area").mean(["latitude", "time"]).cf.plot(x="longitude")
ds.air.mean(["lat", "time"]).cf.plot(x="longitude")